In [ ]:
%%time
import requests
import time
import json
import random
from fake_useragent import UserAgent
from urllib import parse

class TencentSpider(object):
    def __init__(self):
        self.one_url = 'https://careers.tencent.com/tencentcareer/api/post/Query?timestamp=1571206957525&countryId=&cityId=&bgIds=&productId=&categoryId=&parentCategoryId=&attrId=&keyword={}&pageIndex={}&pageSize=10&language=zh-cn&area=cn'
        self.two_url = 'https://careers.tencent.com/tencentcareer/api/post/ByPostId?timestamp=1571207049293&postId={}&language=zh-cn'
        self.ua = UserAgent()
        # 将所有抓取的数据保存到json文件中
        self.list = []
    
    def parse_html(self,url,num):
        print('---第{}页抓取中---'.format(num))
        one_html = requests.get(
            url=url,
            headers={'User-Agent':self.ua.random}
        ).json()
        # 提取postid,拼接二级页面地址
        if one_html:
            for one in one_html['Data']['Posts']:
                # postId, two_url
                postid = one['PostId']
                two_url = self.two_url.format(postid)
                self.list.append(self.get_data(two_url))
            print('---第{}页抓取成功---'.format(num))
            
    def get_data(self,two_url):
        html = requests.get(
            url=two_url,
            headers={'User-Agent':self.ua.random}
        ).json()
        data = html['Data']
        job = {}
        job['name'] = data['RecruitPostName']
        job['duty'] = data['Responsibility']
        job['require'] = data['Requirement']
        return job
    
    def get_count(self,keyword):
        html = requests.get(
            url=self.one_url.format(keyword,1),
            headers={'User-Agent':self.ua.random},
        ).json()
        return int(html['Data']['Count'])+1
    
    def save_json(self,keyword):
        # 将数据保存到json文件中
        with open('{}.json'.format(keyword),'w') as f:
            print('{}个职位已经成功抓取'.format(len(self.list)))
            json.dump(self.list,f,ensure_ascii=False)
        
    def run(self):
        keyword = input('请输入关键字:')
        keyword_quote = parse.quote(keyword)
        count = self.get_count(keyword_quote)
        for page in range(1,count//10+1):
            url = self.one_url.format(keyword_quote,page)
            self.parse_html(url,page)
            time.sleep(random.random())
        self.save_json(keyword)
            
spider = TencentSpider()
spider.run()